## Colab Configuration and Imports


In [ ]:
import pandas as pd
import json as json
import re

from sklearn import preprocessing

In [ ]:
!wget --no-check-certificate -r "https://drive.google.com/uc?export=download&id=1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt" -O "BRM-emot-submit-val-arousal.csv"
!wget --no-check-certificate -r "https://drive.google.com/uc?export=download&id=1SIbc-hi_SJJJEantgYZcSCGNvn-2eHFk" -O "imagenet1000_clsidx_to_labels.json"

will be placed in the single file you specified.

--2021-09-06 20:03:12--  https://drive.google.com/uc?export=download&id=1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt
Resolving drive.google.com (drive.google.com)... 142.250.141.100, 142.250.141.138, 142.250.141.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/r06istkl5c2ma48gaddiigfk735aiamo/1630958550000/16730855647952832235/*/1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt?e=download [following]
--2021-09-06 20:03:15--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/r06istkl5c2ma48gaddiigfk735aiamo/1630958550000/16730855647952832235/*/1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Con

# Map ImageNet Classes to BRM-Word dataset

Looping through ImageNet Classes we search the matching key word in BRM-Words printing it in a new .csv file to feed the Visualizer

In [ ]:
# read datasets
brm_df = pd.read_csv('BRM-emot-submit-val-arousal.csv')

with open('imagenet1000_clsidx_to_labels.json') as json_file:
    imagenet_classes = json.load(json_file)

# print header of csv
with open('imagenet-classes-v-a.csv', 'w') as the_file:
    the_file.write('Id,Class,WordBRM,V.Mean.Sum,A.Mean.Sum\n')

# loop through imagenet classes
for key, value in imagenet_classes.items():

    # split class name in single words
    imagenet_words = re.split(', | ', value)

    print("IMAGENET WORD: " + str(key) + " - " + value)

    # loop through each word
    for imagenet_word in imagenet_words:

        # find the first word that match in BRM dataset
        brm_word = brm_df[brm_df['Word'] == imagenet_word]

        # if found print it in the out file interesting parts
        if not brm_word.empty:
            print(brm_word)

            # prepare the line to print
            string = "{0},{1},{2},{3},{4}\n".format(key, value.replace(",", " |"), brm_word['Word'].values[0],
                                                    brm_word['V.Mean.Sum'].values[0], brm_word['A.Mean.Sum'].values[0])
            with open('imagenet-classes-v-a.csv', 'a') as the_file:
                the_file.write(string)
            break



IMAGENET WORD: 0 - fish
      Unnamed: 0  Word  V.Mean.Sum  ...  D.Mean.H  D.SD.H  D.Rat.H
4664        4665  fish        6.42  ...      5.25    2.71        8

[1 rows x 65 columns]
IMAGENET WORD: 1 - goldfish
      Unnamed: 0      Word  V.Mean.Sum  ...  D.Mean.H  D.SD.H  D.Rat.H
5270        5271  goldfish        6.62  ...      4.33    2.24        9

[1 rows x 65 columns]
IMAGENET WORD: 2 - shark
       Unnamed: 0   Word  V.Mean.Sum  ...  D.Mean.H  D.SD.H  D.Rat.H
11010       11011  shark        4.02  ...       4.7    1.83       10

[1 rows x 65 columns]
IMAGENET WORD: 3 - shark
       Unnamed: 0   Word  V.Mean.Sum  ...  D.Mean.H  D.SD.H  D.Rat.H
11010       11011  shark        4.02  ...       4.7    1.83       10

[1 rows x 65 columns]
IMAGENET WORD: 4 - shark
       Unnamed: 0   Word  V.Mean.Sum  ...  D.Mean.H  D.SD.H  D.Rat.H
11010       11011  shark        4.02  ...       4.7    1.83       10

[1 rows x 65 columns]
IMAGENET WORD: 5 - torpedo
       Unnamed: 0     Word  V.Mean.Sum  .

#Normalization
Normalize the ImageNet classes Valence-Arousal values to the range [-1 , 1] so to match the CNN model predictions values range.


In [ ]:
brm_df = pd.read_csv('/content/imagenet-classes-v-a.csv')
brm_df.describe()

,Id,V.Mean.Sum,A.Mean.Sum
count,981.000000,981.000000,981.000000
mean,498.069317,5.649541,4.111193
std,288.121405,0.991262,0.962170
min,0.000000,1.620000,2.290000
25%,249.000000,5.100000,3.480000
50%,497.000000,5.740000,3.910000
75%,745.000000,6.360000,4.650000
max,999.000000,7.890000,7.240000


In [ ]:
#normalize [-1,1]
scaler=preprocessing.MinMaxScaler(feature_range=(-1,1))
brm_df.iloc[:,3:5]=scaler.fit_transform(brm_df.iloc[:,3:5])
brm_df.describe()

,Id,V.Mean.Sum,A.Mean.Sum
count,981.000000,981.000000,981.000000
mean,498.069317,0.285340,-0.264165
std,288.121405,0.316192,0.388756
min,0.000000,-1.000000,-1.000000
25%,249.000000,0.110048,-0.519192
50%,497.000000,0.314195,-0.345455
75%,745.000000,0.511962,-0.046465
max,999.000000,1.000000,1.000000


Save the scaled version to .csv file

In [ ]:
brm_df.to_csv('/content/imagenet-classes-v-a_scaled.csv', index=False)